In [1]:
import time
import sqlalchemy, os
import pandas as pd
from binance.client import Client
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")
client = Client(API_KEY, API_SECRET)

In [7]:
def function(list_of_symbols):
    ls={}
    global total_profit
    cumulative_profit=0.0
    for symbol in list_of_symbols:
        first_avg_price = float(client.get_avg_price(symbol=symbol)['price'])
        ls[symbol] = first_avg_price
        
    time.sleep(10)
    
    for symbol in list_of_symbols:
        second_avg_price = float(client.get_avg_price(symbol=symbol)['price'])
        first_avg_price = ls[symbol]
        profit = ((second_avg_price - first_avg_price)/first_avg_price)*100
        
        cumulative_profit+=profit
        print(symbol, str(profit)+'%')
    total_profit += cumulative_profit
    print('Cumulative Profit: '+str(cumulative_profit)+'%\n')
    print('Total Profit: '+str(total_profit)+'%\n')

def start():
    with open('pairs_level_3.txt','r') as file:
        file.seek(0) # Ensure you're at the start of the file..
        first_char = file.read(1) # Get the first character
        if not first_char:
            print("file is empty") # The first character is the empty string..
            time.sleep(10)
        else:
            file.seek(0) # The first character wasn't empty. Return to the start of the file.
             # Use file now
            function(file.readline().split(','))
            
while(True):
    start()
    time.sleep(5)

DOCKUSDT -0.006991417602567%
Cumulative Profit: -0.006991417602567%

Total Profit: -0.006991417602567%

DOCKUSDT -0.004175706824855876%
Cumulative Profit: -0.004175706824855876%

Total Profit: -0.011167124427422875%

DOCKUSDT -0.046255914802139454%
Cumulative Profit: -0.046255914802139454%

Total Profit: -0.05742303922956233%

DOCKUSDT 0.0%
Cumulative Profit: 0.0%

Total Profit: -0.05742303922956233%

DOCKUSDT 0.0019515742781534662%
Cumulative Profit: 0.0019515742781534662%

Total Profit: -0.055471464951408866%

TRXUPUSDT 0.013741619591413264%
DOCKUSDT 0.01921841806236312%
Cumulative Profit: 0.032960037653776385%

Total Profit: -0.02251142729763248%

TRXUPUSDT -0.04292175011429612%
DOCKUSDT 0.03663362645995018%
Cumulative Profit: -0.006288123654345945%

Total Profit: -0.028799550951978427%

TRXUPUSDT -0.00037971357446743483%
DOCKUSDT 0.00975451226449798%
Cumulative Profit: 0.009374798690030547%

Total Profit: -0.01942475226194788%

TRXUPUSDT 0.0%
Cumulative Profit: 0.0%

Total Profit: 

KeyboardInterrupt: 

### Profit Window (1%)

In [11]:
def function(list_of_symbols):
    for symbol in list_of_symbols:
        if symbol not in inventory:
            inventory[symbol] = float(client.get_avg_price(symbol=symbol)['price'])
            
    for symbol in inventory:
        price = float(client.get_avg_price(symbol=symbol)['price'])
        if price > inventory[symbol]*1.01:
            total_profit += ((price-inventory[symbol])/inventory[symbol])*100
            inventory.pop(symbol)
            
    print(inventory.keys())
    print('Total Profit: '+str(total_profit)+'%\n')

def start():
    with open('test.txt','r') as file:
        file.seek(0) # Ensure you're at the start of the file..
        first_char = file.read(1) # Get the first character
        if not first_char:
            u=0 # The first character is the empty string..
        else:
            file.seek(0) # The first character wasn't empty. Return to the start of the file.
             # Use file now
            function(file.readline().split(','))

print("Program starts")
global inventory
global total_profit
inventory['x']=0
inventory.pop('x')
total_profit=0
while(True):
    start()

Program starts


NameError: name 'inventory' is not defined